In [1]:
import pandas as pd
import numpy as np
import regex as re

from useful_functions import *

In [2]:
q1_date_df = pickle_loader("q1_date_df")

### Part 1: Quick and Dirty Analysis for creating article loop:

#### 1.1 - 3 Main problem areas: (Copied from 7e)

1- Publ. year of article not matching with publ. year of jrnl -> 3

2- Subm. date is later than publ. date -> 3514

3- No author (cntry) info 

In [ ]:
# TDD 1:

q1_date_df["publ_year"] = q1_date_df.published.map(lambda x: x["date-parts"][0][0])

In [ ]:
q1_date_df[q1_date_df.publ_year != q1_date_df.JRNL_YEAR]

# 3 articles!

In [ ]:
# TTD 2:
q1_date_df["Submitted_Date"] = q1_date_df.Dates.map(lambda x: x[0])
q1_date_df["Accepted_Date"] = q1_date_df.Dates.map(lambda x: x[1])

q1_date_df["Published_Date"] = q1_date_df.published.map(lambda x: x["date-parts"][0])

In [ ]:
def simpl_dt_converter(sub_d):
    try:
        return pd.to_datetime(sub_d, infer_datetime_format=True)
    except:
        return np.nan

In [ ]:
q1_date_df["Submitted_datetime"] = q1_date_df.Submitted_Date.map(simpl_dt_converter)

q1_date_df["Published_datetime"] = q1_date_df.Published_Date.map(lambda x: "-".join([str(it) for it in x])).map(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

In [ ]:
q1_date_df[q1_date_df.Submitted_datetime > q1_date_df.Published_datetime]

# 3514 articles!

In [2]:
# TDD 3:
q1_first_author_df.loc[:,"index"] = q1_first_author_df.index.map(lambda x: "aut_" + str(x))
q1_first_author_df.set_index("index", inplace=True)

#  Creating a auth & doi mapping dict w/ only valid authors considered:

q1_first_author_df.loc[q1_first_author_df.country.notna(),"consider"] = q1_first_author_df[q1_first_author_df.country.notna()].country.map(lambda x: True if len(x)>0 else False)
q1_first_author_df.consider.fillna(False, inplace=True)

doi_aut_dict = dict()

for aut_id, row in q1_first_author_df[q1_first_author_df.consider == True].iterrows():
    doi_list = row["DOI"]
    for doi in doi_list:
        if doi in doi_aut_dict.keys():
            doi_aut_dict[doi].append(aut_id)
        else:
            doi_aut_dict[doi] = [aut_id]


q1_date_df_wip["AUTH_ID"] = q1_date_df_wip.index.map(doi_aut_dict)

In [14]:
pickle_dumper(q1_date_df_wip, "q1_date_df_wip")

In [10]:
pickle_dumper(q1_first_author_df, "q1_first_author_df")

In [10]:
# Combining 3 conditions - 3 MASKS:

year_mask = q1_date_df_wip.publ_year == q1_date_df_wip.JRNL_YEAR
subm_publ_mask = q1_date_df_wip.Submitted_datetime < q1_date_df_wip.Published_datetime
auth_mask = q1_date_df_wip.AUTH_ID.notna()

In [12]:
len(q1_date_df_wip[year_mask & subm_publ_mask & auth_mask])

172992

### Part 2: Pivots & Loops for SumCet & Ogi runs

With the main 3 conditions filtered, we end up with 172992 articles.

However, earlier roday as the author masking took longer than expected, the dataset sent to SummCet only considered auth mask and not the 1st & 2nd masks.

As a solution, will create a Ogi CR+SH loop with only these 2 masks considered.


#### 2.1 - Pivots:

In [15]:
# Ogi Pivot:

current_pivot = pd.pivot_table(data=q1_date_df_wip[year_mask & subm_publ_mask], columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")
q1_a_ss = pickle_loader("7b_variables/q1_a_ss")
missing_pivot = (q1_a_ss - current_pivot).applymap(lambda x: x if x>0 else 0)

missing_pivot

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,2,12,14,0,0,0,0,0,0,0,0
"Biochemistry, Genetics and Molecular Biology",0,0,1,0,0,0,0,0,0,0,0
"Business, Management and Accounting",0,0,0,0,0,0,0,0,0,0,0
Chemical Engineering,0,6,0,0,0,0,0,0,2,0,0
Chemistry,0,0,0,0,0,0,0,0,0,0,4
Computer Science,0,6,0,0,0,0,0,0,0,0,0
Decision Sciences,0,0,0,0,0,0,0,0,0,0,0
Dentistry,33,34,9,31,74,71,51,65,0,6,14


In [190]:
# SumCet pivot:

current_pivot = pd.pivot_table(data=q1_date_df_wip, columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")
q1_a_ss = pickle_loader("7b_variables/q1_a_ss")
missing_pivot = (q1_a_ss - current_pivot).applymap(lambda x: x if x>0 else 0)

missing_pivot

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,52,31,0,5,0,10,6,13,0,0,28
"Biochemistry, Genetics and Molecular Biology",0,0,10,0,0,0,0,0,0,0,5
"Business, Management and Accounting",0,14,0,12,30,7,19,18,9,0,0
Chemical Engineering,0,5,0,12,15,0,12,0,18,28,15
Chemistry,0,6,1,1,21,13,30,29,35,12,39
Computer Science,5,18,5,4,14,21,0,0,18,5,20
Decision Sciences,0,8,0,12,0,0,13,0,11,8,28
Dentistry,34,35,16,44,69,45,19,74,14,34,30


#### 2.2 Run Preps:

These cells were run in seperate times. So, the same cells were used for SumCet & Ogi runs.

In [18]:
q1_jrnl_df_wip = pickle_loader("q1_jrnl_df")

In [20]:
# Creating empty columns:

q1_jrnl_df_wip["Required_Count"] = q1_jrnl_df_wip.Sbj_Area.map(dict)
q1_jrnl_df_wip["Required_Count"] = q1_jrnl_df_wip.Required_Count.map(lambda x: dict.fromkeys(x,int(0)))

q1_jrnl_df_wip["SA_list"] = q1_jrnl_df_wip.Required_Count.map(lambda x: list(x.keys()))

q1_jrnl_df_wip["Required_Count_Sum"] = 0

In [21]:
# Distributing missing articles to journals:

multip = 1.5

for s_a, row in missing_pivot.iterrows(): #iterate over rows
    print(s_a)
    for year, value in row.items():
        # print(year)
        if value == 0:
            pass
        else:
            for i in range(round(value*multip)):
                # select one: 
                sampl_index = q1_jrnl_df_wip[(q1_jrnl_df_wip.Consider == True) & (q1_jrnl_df_wip.CR_Count > (q1_jrnl_df_wip.Required_Count_Sum + q1_jrnl_df_wip.Current_Count)) & (q1_jrnl_df_wip.SA_list.map(lambda x: s_a in x)) & (q1_jrnl_df_wip.Year == year)].sample(1).index.item()


                q1_jrnl_df_wip.loc[sampl_index,"Required_Count"][s_a] += 1
                q1_jrnl_df_wip.loc[sampl_index,"Required_Count_Sum"] += 1

Agricultural and Biological Sciences
Arts and Humanities
Biochemistry, Genetics and Molecular Biology
Business, Management and Accounting
Chemical Engineering
Chemistry
Computer Science
Decision Sciences
Dentistry
Earth and Planetary Sciences
Economics, Econometrics and Finance
Energy
Engineering
Environmental Science
Health Professions
Immunology and Microbiology
Materials Science
Mathematics
Medicine
Multidisciplinary
Neuroscience
Nursing
Pharmacology, Toxicology and Pharmaceutics
Physics and Astronomy
Psychology
Social Sciences
Veterinary


In [196]:
# SumCet:
q1_jrnl_df_wip.Required_Count_Sum.sum()

4898

In [22]:
# Ogi:
q1_jrnl_df_wip.Required_Count_Sum.sum()

2233

In [23]:
remainder_df = pickle_loader("q1_remainder_df")
# DOI list to be skipped in the loop:
all_dois = set(q1_date_df_wip.index.to_list() + remainder_df.index.to_list())

pickle_dumper(filename="all_dois",variable=all_dois)


In [24]:
# Filtered version of jrnl_df to be used in the loop:

required_df = q1_jrnl_df_wip[q1_jrnl_df_wip.Required_Count_Sum > 0]

pickle_dumper(filename="required_df_Ogi",variable=required_df)



### NOTE:

These three masks were used to create a final SumCet & Ogi loop for any possible problematic articles.

During main analysis there will most definitely more issues with some articles. However, these 3 are the most basic ones to date. Therefore, this nb will be used to get & combine the results, ie. as a workbench. The main analysis will be done on a new nb with some additional fixes to the steps above.

### Part 3: q1_date_df_wip & q1_first_author_df - Analysis & Graphing

In [2]:
q1_date_df_wip = pickle_loader("q1_date_df_wip")

In [3]:
q1_first_author_df = pickle_loader("q1_first_author_df")

In [6]:
# Combining 3 conditions - 3 MASKS:

year_mask = q1_date_df_wip.publ_year == q1_date_df_wip.JRNL_YEAR
subm_publ_mask = q1_date_df_wip.Submitted_datetime < q1_date_df_wip.Published_datetime
auth_mask = q1_date_df_wip.AUTH_ID.notna()

In [ ]:
q1_date_df_wip[year_mask & subm_publ_mask & auth_mask]

In [5]:
q1_date_df_wip.AUTH_ID.isna().value_counts()

False    176841
True       9661
Name: AUTH_ID, dtype: int64

TTD:

- Consider col in first_author only considers it's own issues. However due to the mask, there are some authors that are now useless. Need to filter them out as well.


In [4]:
q1_date_df_wip.columns

Index(['JRNL_ID', 'ISSN', 'issued', 'container-title', 'publisher', 'title',
       'author', 'subject', 'references-count', 'is-referenced-by-count',
       'published', 'published-print', 'published-online', 'direct_url',
       'Dates', 'Keywords', 'Chosen_SA', 'JRNL_YEAR', 'publ_year',
       'Submitted_Date', 'Accepted_Date', 'Submitted_datetime',
       'Published_Date', 'Published_datetime', 'AUTH_ID'],
      dtype='object')

In [17]:
q1_first_author_df[q1_first_author_df.consider == True].country.map(len).value_counts().sort_index()

1     130849
2      15643
3       5366
4       2917
5       1884
6       1331
7       1030
8        801
9        655
10       541
11       491
12       453
13       375
14       337
15       296
16       247
17       254
18       208
19       259
20       212
21       174
22       193
23       181
24       227
25      4844
Name: country, dtype: int64

In [21]:
q1_first_author_df["country_counter"] = q1_first_author_df[q1_first_author_df.consider == True].country.map(Counter)

In [27]:
q1_first_author_df["first_country"] = q1_first_author_df[q1_first_author_df.consider == True].country.map(lambda x: x[0])

In [34]:
q1_first_author_df["most_common_country"] = q1_first_author_df[q1_first_author_df.consider == True].country_counter.map(lambda x: x.most_common()[0][0])

In [36]:
q1_first_author_df

,new_given,new_family,SA_list,DOI,country,given,family,doc_count,consider,country_counter,first_country,most_common_country
index,,,,,,,,,,,,
aut_0,jing,cai,"[Social Sciences, Arts and Humanities]",[10.1016/j.jeap.2016.09.002],"[China, China, China, Hong Kong]","[Jing, Jingheng, Jing, (Luna) Jing]","[Cai, Cai, Cai, Cai]","[23, 22, 1, 1]",True,"{'China': 3, 'Hong Kong': 1}",China,China
aut_1,qing,hao,"[Economics, Econometrics and Finance]",[10.1016/j.finmar.2015.11.003],"[United States, United States, United States, ...","[Qing, Qing, (Grace) Qing, Grace Qing]","[Hao, Hao, Hao, Hao]","[86, 11, 2, 1]",True,{'United States': 4},United States,United States
aut_2,yong,zhou,"[Business, Management and Accounting]",[10.1080/10548408.2015.1075933],"[China, China, China, China, China, China, Chi...","[Yongquan, Yonghong, Yong, Dayong, Yongwu, Yon...","[Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zho...","[286, 220, 193, 170, 164, 151, 100, 92, 74, 64...",True,"{'China': 23, 'Japan': 1, 'Hong Kong': 1}",China,China
aut_3,A.,abba,"[Engineering, Energy]",[10.1109/tns.2010.2049658],"[Italy, Italy, France, Nigeria, Malaysia, Saud...","[Andrea, Alessandro, Ado Adamou Abba, Hafizull...","[Abba, Abbà, Ari, Ahmed, Haruna, Abba, Abba, A...","[75, 65, 44, 19, 17, 15, 8, 3]",True,"{'Italy': 2, 'France': 1, 'Nigeria': 2, 'Malay...",Italy,Italy
aut_4,A.,agodi,"[Medicine, Nursing]",[10.1038/ejcn.2011.125],[Italy],[Antonella],[Agodi],[234],True,{'Italy': 1},Italy,Italy
...,...,...,...,...,...,...,...,...,...,...,...,...
aut_176391,zorana,jančić,"[Computer Science, Engineering, Decision Scien...",[10.1016/j.ins.2010.12.008],[Serbia],[Zorana],[Jančić],[9],True,{'Serbia': 1},Serbia,Serbia
aut_176392,zoë,avner,"[Social Sciences, Medicine, Health Professions]",[10.1080/13573322.2020.1782881],[United Kingdom],[Zoë],[Avner],[7],True,{'United Kingdom': 1},United Kingdom,United Kingdom
aut_176393,zs.,szendro,"[Agricultural and Biological Sciences, Veterin...",[10.1016/j.livsci.2010.11.012],[Hungary],[Zs],[Szendrő],[135],True,{'Hungary': 1},Hungary,Hungary


In [37]:
len(q1_first_author_df[q1_first_author_df.consider == True][q1_first_author_df[q1_first_author_df.consider == True].first_country != q1_first_author_df[q1_first_author_df.consider == True].most_common_country])

5763

In [38]:
(q1_first_author_df[q1_first_author_df.consider == True][q1_first_author_df[q1_first_author_df.consider == True].first_country != q1_first_author_df[q1_first_author_df.consider == True].most_common_country]).sample(10)

,new_given,new_family,SA_list,DOI,country,given,family,doc_count,consider,country_counter,first_country,most_common_country
index,,,,,,,,,,,,
aut_23273,c. y.,tang,"[Agricultural and Biological Sciences, Mathema...",[10.1093/biomet/ass045],"[Taiwan, Japan, China, United States, United S...","[Chuanyi, Changyuan, Chaoying, Cheng Yong, Yuc...","[Tang, Tang, Tang, Tang, Tang, Tang]","[209, 152, 38, 36, 27, 18]",True,"{'Taiwan': 1, 'Japan': 1, 'China': 2, 'United ...",Taiwan,China
aut_3572,N. P.,landsman,[Arts and Humanities],[10.1016/j.shpsb.2013.07.003],"[None, Netherlands]","[Nicolaas P.Klaas, Klaas]","[Landsman, Landsman]","[54, 26]",True,"{None: 1, 'Netherlands': 1}",None,None
aut_33101,david,hill,"[Arts and Humanities, Psychology]",[10.1016/j.intell.2011.06.005],"[Australia, Australia, United States, United S...","[Keith D., David John, William David, David L....","[Hill, Hill, Hill, Hill, Hill, Hill, Hill, Hil...","[368, 202, 106, 81, 60, 46, 22, 10, 7, 5, 4, 1...",True,"{'Australia': 2, 'United States': 8, 'United K...",Australia,United States
aut_143164,yue,cao,[Engineering],[10.1007/s00163-020-00334-9],"[United States, China, China, China, China, Ch...","[Yue, Yue, Rongyue, Yueqiang, Yuelong, Yue, Yu...","[Cao, Cao, Cao, Cao, Cao, Cao, Cao, Cao, Cao, ...","[190, 150, 84, 51, 48, 37, 37, 33, 32, 32, 31,...",True,"{'United States': 4, 'China': 19, 'Australia':...",United States,China
aut_106917,q.,yang,"[Materials Science, Engineering, Physics and A...","[10.1007/s00161-017-0557-y, 10.1016/j.nonrwa.2...","[United States, China, China, China, China, Ch...","[Liuqing, Qiang, Qi, Jiquan, Yongqing, Qing, J...","[Yang, Yang, Ouyang, Yang, Yang, Yang, Yang, Y...","[416, 294, 279, 135, 131, 105, 90, 28, 5]",True,"{'United States': 1, 'China': 8}",United States,China
aut_22753,c.,finch,[Environmental Science],[10.1002/rra.2893],"[Australia, United States, United States, Unit...","[C. F., Caleb Ellicott, Clement A.C.A., Stuart...","[Finch, Finch, Finch, Finch, Finch, Finch, Bak...","[446, 438, 320, 136, 58, 33, 27, 24, 6, 6, 5, ...",True,"{'Australia': 1, 'United States': 14, 'New Zea...",Australia,United States
aut_105086,peter,wei,[Computer Science],[10.1145/3218582],"[New Zealand, United States, United States, Un...","[Weiliang, Peter, Peter, Peter J., Peter]","[Xu, Wei, Wei, Wei, Wei]","[334, 18, 3, 2, 1]",True,"{'New Zealand': 1, 'United States': 4}",New Zealand,United States
aut_138685,xinnan,wang,"[Biochemistry, Genetics and Molecular Biology]",[10.1016/j.cell.2011.10.018],"[United States, United States, China, Hong Kon...","[Xinnan, Xinnan, Xinnan, Xinnan, Xinnan, Xinna...","[Wang, Wang, Wang, Wang, Wang, Wang, Wang, Wan...","[38, 31, 23, 6, 5, 5, 3, 2, 2, 1, 1]",True,"{'United States': 3, 'China': 6, 'Hong Kong': ...",United States,China
aut_93987,min,sun,"[Chemistry, Chemical Engineering, Energy, Phys...","[10.1016/j.talanta.2012.11.077, 10.1016/j.pecs...","[United States, China, China, China, China, Ch...","[Mingui, Weimin, Baomin, Guangmin, Junmin, Min...","[Sun, Sun, Sun, Sun, Sun, Sun, Sun]","[407, 298, 205, 154, 56, 42, 7]",True,"{'United States': 1, 'China': 6}",United States,China


There are 5763 authors that hae different first and most common countries.

In these cases we can use a function to check the first indexes of the exact names & surnames. However, most of the names here are abbrev. & we don't have an exact match for the name. 

In [18]:
from collections import Counter

In [9]:
q1_first_author_df.consider.value_counts()

True     169768
False      6628
Name: consider, dtype: int64

Both are ready & running.

Tomorrow:

- collect SumCet & Ogi results
    - need to combine both required_dfs to correctly assign SA to new articles :( - NOPE!
        USE their respective required_dfs instead!
- Scopus loop for SumCetOgi
- use q1_date_df_wip & masks to start graphing and analyzing
- add all once Scopus is complete!

----

### Testing Area:

In [116]:
(q1_first_author_df.dropna(subset="country").country.map(len) > 0).sum()

169768

In [10]:
#Trial 1:
# q1_first_authkor_df.loc[q1_first_author_df.country.notna(),"consider"] = q1_first_author_df[q1_first_author_df.country.notna()].country.map(lambda x: True if len(x)>0 else False)
q1_first_author_df.consider.fillna(False, inplace=True)

# aut_doi_dict = dict()

# for aut_id, row in q1_first_author_df[q1_first_author_df.consider == True].iterrows():
#     doi_list = row["DOI"]
#     for doi in doi_list:
#         if doi in aut_doi_dict.keys():
#             aut_doi_dict[doi].append(aut_id)
#         else:
#             aut_doi_dict[doi] = [aut_id]


In [11]:
q1_first_author_df[q1_first_author_df.country.notna()].consider.value_counts()

True     169768
False      6618
Name: consider, dtype: int64

In [222]:
q1_first_author_df.consider()

176396

In [213]:
len(q1_first_author_df)

176396

In [215]:
q1_first_author_df

,new_given,new_family,SA_list,DOI,country,given,family,doc_count,consider
index,,,,,,,,,
aut_0,jing,cai,"[Social Sciences, Arts and Humanities]",[10.1016/j.jeap.2016.09.002],"[China, China, China, Hong Kong]","[Jing, Jingheng, Jing, (Luna) Jing]","[Cai, Cai, Cai, Cai]","[23, 22, 1, 1]",True
aut_1,qing,hao,"[Economics, Econometrics and Finance]",[10.1016/j.finmar.2015.11.003],"[United States, United States, United States, ...","[Qing, Qing, (Grace) Qing, Grace Qing]","[Hao, Hao, Hao, Hao]","[86, 11, 2, 1]",True
aut_2,yong,zhou,"[Business, Management and Accounting]",[10.1080/10548408.2015.1075933],"[China, China, China, China, China, China, Chi...","[Yongquan, Yonghong, Yong, Dayong, Yongwu, Yon...","[Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zho...","[286, 220, 193, 170, 164, 151, 100, 92, 74, 64...",True
aut_3,A.,abba,"[Engineering, Energy]",[10.1109/tns.2010.2049658],"[Italy, Italy, France, Nigeria, Malaysia, Saud...","[Andrea, Alessandro, Ado Adamou Abba, Hafizull...","[Abba, Abbà, Ari, Ahmed, Haruna, Abba, Abba, A...","[75, 65, 44, 19, 17, 15, 8, 3]",True
aut_4,A.,agodi,"[Medicine, Nursing]",[10.1038/ejcn.2011.125],[Italy],[Antonella],[Agodi],[234],True
...,...,...,...,...,...,...,...,...,...
aut_176391,zorana,jančić,"[Computer Science, Engineering, Decision Scien...",[10.1016/j.ins.2010.12.008],[Serbia],[Zorana],[Jančić],[9],True
aut_176392,zoë,avner,"[Social Sciences, Medicine, Health Professions]",[10.1080/13573322.2020.1782881],[United Kingdom],[Zoë],[Avner],[7],True
aut_176393,zs.,szendro,"[Agricultural and Biological Sciences, Veterin...",[10.1016/j.livsci.2010.11.012],[Hungary],[Zs],[Szendrő],[135],True


In [218]:
len(q1_first_author_df.iloc[4]["country"])

1

In [170]:
len(aut_doi_dict)

176841

Something is wrong with this dict, causing lot less authors than expected, don't know what -> EDIT: NOT ACTUALLY!

In [ ]:
# Working "bok" solution:

q1_first_author_df.country.fillna("bok", inplace=True)

q1_first_author_df.loc[q1_first_author_df.country.map(lambda x: True if len(x) ==  0 else False),"country"] = "bok"

q1_first_author_df_clean = q1_first_author_df[q1_first_author_df.country.map(type) == list].copy()

doi_aut_dict = q1_first_author_df_clean.explode("DOI").reset_index().groupby("DOI")["index"].apply(list).to_dict()

# q1_date_df_wip["AUTH_ID"] = q1_date_df.index.map(doi_aut_dict)

# q1_date_df_wip = q1_date_df_wip[q1_date_df_wip.AUTH_ID.notna()]

In [75]:
#  Trial 2:
aut_doi_dict2 = dict()

for aut_id, row in q1_first_author_df.iterrows():
    try:
        doi_list = row["DOI"]
        for doi in doi_list:
            if doi in aut_doi_dict2.keys():
                aut_doi_dict2[doi].append(aut_id)
            else:
                aut_doi_dict2[doi] = [aut_id]
    except:
        pass

In [208]:
q1_first_author_df

,new_given,new_family,SA_list,DOI,country,given,family,doc_count
index,,,,,,,,
aut_0,jing,cai,"[Social Sciences, Arts and Humanities]",[10.1016/j.jeap.2016.09.002],"[China, China, China, Hong Kong]","[Jing, Jingheng, Jing, (Luna) Jing]","[Cai, Cai, Cai, Cai]","[23, 22, 1, 1]"
aut_1,qing,hao,"[Economics, Econometrics and Finance]",[10.1016/j.finmar.2015.11.003],"[United States, United States, United States, ...","[Qing, Qing, (Grace) Qing, Grace Qing]","[Hao, Hao, Hao, Hao]","[86, 11, 2, 1]"
aut_2,yong,zhou,"[Business, Management and Accounting]",[10.1080/10548408.2015.1075933],"[China, China, China, China, China, China, Chi...","[Yongquan, Yonghong, Yong, Dayong, Yongwu, Yon...","[Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zho...","[286, 220, 193, 170, 164, 151, 100, 92, 74, 64..."
aut_3,A.,abba,"[Engineering, Energy]",[10.1109/tns.2010.2049658],"[Italy, Italy, France, Nigeria, Malaysia, Saud...","[Andrea, Alessandro, Ado Adamou Abba, Hafizull...","[Abba, Abbà, Ari, Ahmed, Haruna, Abba, Abba, A...","[75, 65, 44, 19, 17, 15, 8, 3]"
aut_4,A.,agodi,"[Medicine, Nursing]",[10.1038/ejcn.2011.125],[Italy],[Antonella],[Agodi],[234]
...,...,...,...,...,...,...,...,...
aut_176391,zorana,jančić,"[Computer Science, Engineering, Decision Scien...",[10.1016/j.ins.2010.12.008],[Serbia],[Zorana],[Jančić],[9]
aut_176392,zoë,avner,"[Social Sciences, Medicine, Health Professions]",[10.1080/13573322.2020.1782881],[United Kingdom],[Zoë],[Avner],[7]
aut_176393,zs.,szendro,"[Agricultural and Biological Sciences, Veterin...",[10.1016/j.livsci.2010.11.012],[Hungary],[Zs],[Szendrő],[135]


In [209]:
aut_doi_dict3 = dict()

for aut_id, row in q1_first_author_df.iterrows():
    try:
        doi_list = row["DOI"]
        if len(row["country"]) > 0:
            for doi in doi_list:
                if doi in aut_doi_dict3.keys():
                    aut_doi_dict3[doi].append(aut_id)
                else:
                    aut_doi_dict3[doi] = [aut_id]
    except:
        pass

In [81]:
len(aut_doi_dict)

176841

In [76]:
len(aut_doi_dict2)

185648

In [210]:
len(aut_doi_dict3)

185648

In [18]:
q1_first_author_df.loc[:,"index"] = q1_first_author_df.index.map(lambda x: "aut_" + str(x))
q1_first_author_df.set_index("index", inplace=True)

In [26]:
aut_doi_dict = dict()

for aut_id, row in q1_first_author_df[q1_first_author_df.consider == True].iterrows():
    doi_list = row["DOI"]
    for doi in doi_list:
        if doi in aut_doi_dict.keys():
            aut_doi_dict[doi].append(aut_id)
        else:
            aut_doi_dict[doi] = [aut_id]


In [83]:
q1_date_df_wip["AUTH_ID"] = q1_date_df_wip.index.map(aut_doi_dict2)

In [88]:
q1_date_df_wip.AUTH_ID.notna().value_counts()

True     185648
False       854
Name: AUTH_ID, dtype: int64

In [92]:
testo = q1_first_author_df.DOI.map(len).value_counts().reset_index()

In [96]:
(testo["index"] * testo["DOI"]).sum()

190228

In [108]:
testo2 = q1_first_author_df.dropna(subset="country")[q1_first_author_df.dropna(subset="country").country.map(len) >0].DOI.map(len).value_counts().reset_index()

In [122]:
q1_first_author_df.dropna(subset="country")[q1_first_author_df.dropna(subset="country").country.map(len) >0].explode("DOI").reset_index().groupby("DOI")["index"].apply(list)

DOI
10.1002/1438-390x.1005       [aut_98865]
10.1002/1873-3468.12216      [aut_85746]
10.1002/1873-3468.12233     [aut_101154]
10.1002/1873-3468.13587      [aut_42158]
10.1002/1878-0261.12399      [aut_69839]
                                ...     
10.9745/ghsp-d-17-00407      [aut_52354]
10.9745/ghsp-d-18-00229      [aut_83779]
10.9745/ghsp-d-20-00008      [aut_47723]
10.9770/jesi.2018.5.4(6)     [aut_94683]
10.9770/jesi.2019.7.2(8)     [aut_59373]
Name: index, Length: 176841, dtype: object

In [123]:
len(q1_date_df)

186502

In [128]:
q1_first_author_df.dropna(subset="country")[q1_first_author_df.dropna(subset="country").country.map(len) >0].DOI.map(len).sum()

180994

In [110]:
testo2

,index,DOI
0,1,161273
1,2,7018
2,3,995
3,4,230
4,5,99
5,6,50
6,7,33
7,8,16
8,11,12
9,9,12


In [109]:
(testo2["index"] * testo2["DOI"]).sum()

180994

In [133]:
q1_first_author_df.dropna(subset="country")[q1_first_author_df.dropna(subset="country").country.map(len) == 0].explode("DOI").reset_index().groupby("DOI")["index"].apply(list)

DOI
10.1002/2014ms000318      [aut_137728]
10.1002/ab.20342           [aut_27356]
10.1002/ab.20354          [aut_162362]
10.1002/ab.21869          [aut_142784]
10.1002/acs.1177           [aut_80774]
                              ...     
10.7554/elife.31262        [aut_92768]
10.7554/elife.31724       [aut_130239]
10.7554/elife.35037       [aut_135311]
10.7566/jpsj.83.124501    [aut_140048]
10.7567/apex.10.032102    [aut_149965]
Name: index, Length: 9064, dtype: object

In [134]:
len(q1_date_df)

186502

In [135]:
186502 -176841

9661

In [136]:
q1_first_author_df[q1_first_author_df.country.isna()]

,new_given,new_family,SA_list,DOI,country,given,family,doc_count,consider
index,,,,,,,,,
aut_4915,V. L.,castroagud�n,[Agricultural and Biological Sciences],[10.3767/003158516x692149],NaN,NaN,NaN,NaN,False
aut_14163,andr� c.,pieterse,[Arts and Humanities],[10.4102/ve.v36i1.1420],NaN,NaN,NaN,NaN,False
aut_18315,angela,frana§a,"[Materials Science, Biochemistry, Genetics and...",[10.1002/smll.200901006],NaN,NaN,NaN,NaN,False
aut_23897,carlos,capistran,"[Business, Management and Accounting, Economic...",[10.1111/j.1538-4616.2009.00280.x],NaN,NaN,NaN,NaN,False
aut_48200,gizem,h�l�r,[Medicine],[10.1159/000371757],NaN,NaN,NaN,NaN,False
aut_112348,roc��o,perez-iglesias,[Medicine],[10.1176/appi.ajp.2009.09050716],NaN,NaN,NaN,NaN,False
aut_124149,stephan,k�hler,"[Psychology, Medicine]",[10.1159/000369847],NaN,NaN,NaN,NaN,False
aut_151904,e. l .,los,"[Neuroscience, Biochemistry, Genetics and Mole...",[10.1111/j.1365-2826.2010.01983.x],NaN,NaN,NaN,NaN,False
aut_159845,m.,or,"[Agricultural and Biological Sciences, Veterin...",[10.3415/vcot-11-07-0108],NaN,NaN,NaN,NaN,False


In [ ]:
176841